# Assignment 01 solutions


---

## Isomorphic Strings (LC 205)

You are given two strings, `s` and `t`, of the same length. Your task is to determine whether the two strings are **isomorphic**.

Two strings are **isomorphic** if you can replace each character in `s` with another character to obtain `t`, following these rules:

1. Each character in `s` must always map to the **same** character in `t`.
2. Different characters in `s` must map to **different** characters in `t`.
3. A character is allowed to map to itself.
4. The order of characters must remain the same.

In other words, there must be a one-to-one correspondence between the characters of `s` and the characters of `t`.


**Solution: technical note**

To decide whether two strings are isomorphic, we scan them **left to right at the same time** (in lock step). At each position `i`, we look at the pair:

- `s_char = s[i]`
- `t_char = t[i]`

What we need to enforce is a **one-to-one correspondence**:

- if we’ve seen `s_char` before, it must map to the **same** `t_char` as last time
- and if we’ve seen `t_char` before, it must come from the **same** `s_char` as last time

When we see a character again (like the second `'g'` in `"egg"`), we must remember what it mapped to the first time.

Normally, we’d use a dictionary or a set-based structure for fast lookup, but this problem **forbids** that. So we use **two parallel lists** instead:

```python
s_chars_already_seen = []
t_chars_already_seen = []
```

These two lists work like a “pair list”:

- `s_chars_already_seen[j]` and `t_chars_already_seen[j]` are a matched pair
- the **index `j` is the glue** that keeps the mapping together

Example with `s="egg"` and `t="add"`:

After processing the first two positions, we have:

```
s_chars_already_seen = ['e', 'g']
t_chars_already_seen = ['a', 'd']
```

This means:

- `'e' → 'a'`
- `'g' → 'd'`

Suppose we’re at the third character (`'g'` in `s`, `'d'` in `t`).
We need to check whether `'g'` has appeared before in `s_chars_already_seen`.

Because it’s a list, the only way is to **walk through it**:

```python
found = False
j = 0
while j < len(s_chars_already_seen) and not found:
    if s_chars_already_seen[j] == s_char:
        found = True
    j += 1
```

If we find it at some index `j`, then we _must_ verify that the character previously paired with it is exactly today’s `t_char`:

```python
if found:
    if t_chars_already_seen[j] != t_char:
        result = False
```

That is exactly what “consistent mapping” means.

It’s not enough to check **only** “same `s_char` maps to same `t_char`”.

We must also make sure that two _different_ characters in `s` don’t map to the **same** character in `t`.

Example:

```python
s = "ab"
t = "aa"
```

If we only check the `s → t` direction, we might wrongly accept:

- `a → a` (fine)
- `b → a` (looks “consistent” for `b`, but it breaks rule #2)

That’s why the code also scans `t_chars_already_seen` to ensure the reverse pairing is consistent too:

- if `t_char` was seen before, it must come from the same `s_char` as before


In [35]:
def are_isomorphic(t: str, s: str) -> bool:
    """Given two strings, `s` and `t`, of the same length, determine whether
    the two strings are isomorphic. Two strings are isomorphic if we can
    replace each character in `s` with another character to obtain `t`, following
    these rules:

      1. Each character in `s` must always map to the same character in `t`.
      2. Different characters in `s` must map to different characters in `t`.
      3. A character is allowed to map to itself.
      4. The order of characters must remain the same.

    In other words, there must be a one-to-one correspondence between the characters
    of `s` and the characters of `t`.

    An additional constraint is that the problem must be solved with simple
    lists. No sets or other data structures can be used at this point.
    """

    MAX_LENGTH = 500

    # This is the return variable. We use it first to determine if the input
    # data are valid.
    result = (
        s is not None
        and t is not None
        and s.isalpha()
        and t.isalpha()
        and len(s) == len(t)
        and len(s) > 0
        and len(t) <= MAX_LENGTH
    )

    if result:

        # Input data are valid. Now let's get ready to compare the two strings.

        # Characters from strinsg and t that have been already seen as we scan the
        # strings from left to right.
        s_chars_already_seen = []
        t_chars_already_seen = []

        # Loop to iterate over the input strings. The loop runs until the return
        # value becomes false or we exhaust the length of the input strings.
        i = 0
        while i < len(s) and result:

            # Obtain current characters from the input strings
            s_char = s[i]
            t_char = t[i]

            # The loop below determines if the current character from string s
            # has been seen before. If it has, the order in which it is seen
            # (position j) means that the j-th character seen from string t
            # must also match the current character from t. If this is not the
            # case, the strings s and t are not isomorphic. If it is the case,
            # then the strings may be isomorphic up to this point (through
            # their first i characters).
            s_char_seen_before = False
            j = 0
            while j < len(s_chars_already_seen):
                if s_chars_already_seen[j] == s_char:
                    # The current character from s has been seen before;
                    # there will be no need to add it to the list of chars
                    # already seen.
                    s_char_seen_before = True
                    # The order in which the current character from s has
                    # been seen is j. The j-th seen character from string
                    # t must also much the current character from t, for
                    # the strings to be isomorphic.
                    if t_chars_already_seen[j] != t_char:
                        # No match - prepare to disqualify
                        result = False
                # Advance to the element of the list of characters from string s
                j += 1

            # Repeat the loop above for the characters from string t.
            t_char_seen_before = False
            k = 0
            while k < len(t_chars_already_seen):
                if t_chars_already_seen[k] == t_char:
                    t_char_seen_before = True
                    # The order in which the current character from t was
                    # seen before must much the order in which the
                    # current character from s was also seen before. Otherwise
                    # the strings are not isomorphic.
                    if s_chars_already_seen[k] != s_char:
                        result = False
                k += 1

            # If neither character has been seen before, add them
            # to the corresponding "already seen" lists and continue
            # with the next position in the input string (advance i)
            if not s_char_seen_before and not t_char_seen_before:
                s_chars_already_seen.append(s_char)
                t_chars_already_seen.append(t_char)

            # Proceed with the next character from the input strings
            i += 1

    return result

---

## Interleaving Strings (LC 97)

You are given three strings: `s1`, `s2`, and `s3`. Your task is to determine whether `s3` can be formed by **interleaving** the characters of `s1` and `s2`.

An **interleaving** of two strings means:

- All characters from `s1` and `s2` must appear in `s3`.
- The **relative order** of characters from `s1` must be preserved.
- The **relative order** of characters from `s2` must be preserved.
- Characters from `s1` and `s2` may be mixed together one at a time, but you may not reorder characters within either string.

In other words, `s3` is formed by weaving together the characters of `s1` and `s2` while keeping each string’s internal order intact.

### Example

```
Input:  s1 = "abc", s2 = "def", s3 = "adbecf"
Output: True
```

**Explanation:**  
One valid interleaving is:

- Take characters from `s1`: `a`
- Then from `s2`: `d`
- Then from `s1`: `b`
- Then from `s2`: `e`
- Then from `s1`: `c`
- Then from `s2`: `f`

This produces `"adbecf"`, which matches `s3`. The string "daebfc" may also be a valid outcome.

### Example

```
Input:  s1 = "abc", s2 = "def", s3 = "adbefc"
Output: False
```

### Example

```
Input:  s1 = "", s2 = "", s3 = ""
Output: True
```

**Explanation:**  
An empty string can be formed by interleaving two empty strings.

## Constraints

- `0 ≤ len(s1), len(s2) ≤ 100`
- `0 ≤ len(s3) ≤ 200`
- All strings consist of lowercase English letters


In [67]:
def is_interleaved(s1, s2, s3):
    """Determines if s3 is a simple interleaf of strings s1 and s2. A pair of strings may produce
    two interleaves, so we test for both cases. For ilustrative purposes, the examples below use
    both upper and lower case letters.

    Example 1:
    s1 ="ABC" and s2="abc" produce the following two interleaves strings:
    "AaBbCc" and "aAbBcC"

    Example 2:
    In the example above we assume that len(s1) == len(s2). A more general case may apply when
    the lengths are not the same. Strings "abc" and "ABCDE" can also interleaf as:
    "AaBbCcDE" or "aAbBcCDE".

    """

    # Maximum allowed length for input strings
    MAX_LENGTH = 100
    # Interleaving step
    INTERLEAVING = 2

    # Validate data
    valid = (
        s1 is not None
        and s2 is not None
        and s3 is not None
        and len(s1) <= MAX_LENGTH
        and len(s2) <= MAX_LENGTH
        and len(s1) + len(s2) == len(s3)
        and s1.isalpha()
        and s2.isalpha()
    )

    if valid:
        # Data are valid. Now check for interleaving
        # i tracks position in s1 and s2
        i = 0
        # k tracks position in s3
        k = 0
        # We check both possible interleavings, assuming s1 leads or s2 leads.
        # This means that either the first character in s3 comes from s1 or from s2.
        # s1_leads is true when the current interleaving being tested assumes s1
        # provides the first character in s3.
        # s2_leads is true when the current interleaving being tested assumes s2
        # provides the first character in s3.
        s1_leads = True
        s2_leads = True
        # Loop while there are characters in both s1 and s2 to check
        while i < min(len(s1), len(s2)) and (s1_leads or s2_leads):
            s1_leads = s1_leads and (s1[i] == s3[k] and s2[i] == s3[k + 1])
            s2_leads = s2_leads and (s2[i] == s3[k] and s1[i] == s3[k + 1])
            i += 1
            k += INTERLEAVING
        # Now check the remaining characters in s1
        while i < len(s1) and (s1_leads or s2_leads):
            s1_leads = s1_leads and (s1[i] == s3[k])
            s2_leads = s2_leads and (s1[i] == s3[k])
            i += 1
            k += 1
        # Now check the remaining characters in s2
        while i < len(s2) and (s1_leads or s2_leads):
            s1_leads = s1_leads and (s2[i] == s3[k])
            s2_leads = s2_leads and (s2[i] == s3[k])
            i += 1
            k += 1
        # Final validity check: at least one interleaving must be true
        valid = s1_leads or s2_leads
    # Done
    return valid

In [60]:
tests = [
    ["abc", "def", "adbcef", False],
    ["abc", "def", "abdecf", False],
    ["abc", "def", "abcdef", False],
    ["abc", "def", "adbecf", True],
    ["abc", "defghi", "adbecfghi", True],
    ["abc", "defghi", "daebfcghi", True],
    ["abcd", "wxyz", "awbxcydz", True],
    ["a", "bcdedf", "abcdedf", True],
    ["aa", "aa", "aaaa", True],
    ["abc", "xyz", "abcxyz", False],
    ["abcdef", "xyz", "axbyczdef", True],
    ["abc", "def", "adbecf", True],
    ["a", "bc", "aba", False]
]

In [68]:
for test in tests:
    s1, s2, s3, expected = test
    result = is_interleaved(s1, s2, s3)
    print(f'is_interleaved("{s1}", "{s2}", "{s3}") = {result} (expected: {expected})')

is_interleaved("abc", "def", "adbcef") = False (expected: False)
is_interleaved("abc", "def", "abdecf") = False (expected: False)
is_interleaved("abc", "def", "abcdef") = False (expected: False)
is_interleaved("abc", "def", "adbecf") = True (expected: True)
is_interleaved("abc", "defghi", "adbecfghi") = True (expected: True)
is_interleaved("abc", "defghi", "daebfcghi") = True (expected: True)
is_interleaved("abcd", "wxyz", "awbxcydz") = True (expected: True)
is_interleaved("a", "bcdedf", "abcdedf") = True (expected: True)
is_interleaved("aa", "aa", "aaaa") = True (expected: True)
is_interleaved("abc", "xyz", "abcxyz") = False (expected: False)
is_interleaved("abcdef", "xyz", "axbyczdef") = True (expected: True)
is_interleaved("abc", "def", "adbecf") = True (expected: True)
is_interleaved("a", "bc", "aba") = False (expected: False)


---

## Longest Balanced Binary Subarray (LC 525)

You are given a list of integers `nums` that contains only `0`s and `1`s. This is called a **binary array**.

Your task is to find the **maximum length** of a **contiguous subarray** that contains an **equal number of `0`s and `1`s**.

- A **subarray** is a consecutive sequence of elements from the list.
- The subarray must contain the same number of `0`s as `1`s.
- If no such subarray exists, the result should be `0`.


In [ ]:
def contiguous_length(nums):
    """
    Return the maximum length of a contiguous subarray with an equal number
    of 0s and 1s.

    Strategy (brute force / simple approach):
    - Try every possible starting index i
    - For each i, extend a subarray to the right (j = i, i+1, i+2, ...)
    - Count how many 0s and 1s appear in that subarray
    - Every time the counts are equal, update the best length found so far
    - Return the best length found
    """

    # Total number of elements in the array
    n = len(nums)

    # Stores the maximum valid subarray length found so far
    best = 0

    # Outer loop: choose the starting index of the subarray
    i = 0
    while i < n:

        # Reset counters for each new starting position
        zeros = 0
        ones = 0

        # Inner loop: extend the subarray to the right
        j = i
        while j < n:

            # Update counters based on current value
            if nums[j] == 0:
                zeros += 1
            else:
                ones += 1

            # If counts match, we found a valid balanced subarray
            if zeros == ones:

                # Length of current subarray is (j - i + 1)
                length = j - i + 1

                # Update the best answer if this one is longer
                if length > best:
                    best = length

            # Move the end of the subarray to the right
            j += 1

        # Move the start of the subarray to the right
        i += 1

    # Return the maximum balanced subarray length found
    return best